In [1]:
# Dependencies
import requests
from sqlalchemy import create_engine
import pandas as pd
import json
from datetime import datetime as dt

In [2]:
# Save config information.
url = "https://api.covid19api.com/summary"


In [3]:
# Pretty print JSON for all launchpads
response = requests.get(url).json()
#print(json.dumps(response, indent=4, sort_keys=True))


countries = response["Countries"]
print(json.dumps(countries, indent=4, sort_keys=True))



[
    {
        "Country": "ALA Aland Islands",
        "CountryCode": "AX",
        "Date": "2020-04-10T00:43:16Z",
        "NewConfirmed": 0,
        "NewDeaths": 0,
        "NewRecovered": 0,
        "Slug": "ala-aland-islands",
        "TotalConfirmed": 0,
        "TotalDeaths": 0,
        "TotalRecovered": 0
    },
    {
        "Country": "Afghanistan",
        "CountryCode": "AF",
        "Date": "2020-04-10T00:43:16Z",
        "NewConfirmed": 40,
        "NewDeaths": 1,
        "NewRecovered": 3,
        "Slug": "afghanistan",
        "TotalConfirmed": 484,
        "TotalDeaths": 15,
        "TotalRecovered": 32
    },
    {
        "Country": "Albania",
        "CountryCode": "AL",
        "Date": "2020-04-10T00:43:16Z",
        "NewConfirmed": 9,
        "NewDeaths": 1,
        "NewRecovered": 11,
        "Slug": "albania",
        "TotalConfirmed": 409,
        "TotalDeaths": 23,
        "TotalRecovered": 165
    },
    {
        "Country": "Algeria",
        "CountryCode": 

In [4]:
# convert Json file to DataFrame
df = pd.DataFrame.from_dict(countries, orient='columns')
df.head()

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,ALA Aland Islands,AX,ala-aland-islands,0,0,0,0,0,0,2020-04-10T00:43:16Z
1,Afghanistan,AF,afghanistan,40,484,1,15,3,32,2020-04-10T00:43:16Z
2,Albania,AL,albania,9,409,1,23,11,165,2020-04-10T00:43:16Z
3,Algeria,DZ,algeria,94,1666,30,235,110,347,2020-04-10T00:43:16Z
4,American Samoa,AS,american-samoa,0,0,0,0,0,0,2020-04-10T00:43:16Z


In [5]:
# clean up data
df = df.drop('Slug', axis=1)
df.head()

,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-10T00:43:16Z
1,Afghanistan,AF,40,484,1,15,3,32,2020-04-10T00:43:16Z
2,Albania,AL,9,409,1,23,11,165,2020-04-10T00:43:16Z
3,Algeria,DZ,94,1666,30,235,110,347,2020-04-10T00:43:16Z
4,American Samoa,AS,0,0,0,0,0,0,2020-04-10T00:43:16Z


In [6]:
# rename columns
df = df.rename(columns={"Country":"country","CountryCode":"country_code",
                        "NewConfirmed":"new_cases_confirmed","TotalConfirmed":"total_cases_confirmed","NewDeaths":"new_deaths",
                        "TotalDeaths":"total_deaths","NewRecovered":"new_recovered","TotalRecovered":"total_recovered",
                       "Date":"date"})
df.head()

,country,country_code,new_cases_confirmed,total_cases_confirmed,new_deaths,total_deaths,new_recovered,total_recovered,date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-10T00:43:16Z
1,Afghanistan,AF,40,484,1,15,3,32,2020-04-10T00:43:16Z
2,Albania,AL,9,409,1,23,11,165,2020-04-10T00:43:16Z
3,Algeria,DZ,94,1666,30,235,110,347,2020-04-10T00:43:16Z
4,American Samoa,AS,0,0,0,0,0,0,2020-04-10T00:43:16Z


In [7]:
# format date
df['date'] = pd.to_datetime(df['date']).dt.date
df

,country,country_code,new_cases_confirmed,total_cases_confirmed,new_deaths,total_deaths,new_recovered,total_recovered,date
0,ALA Aland Islands,AX,0,0,0,0,0,0,2020-04-10
1,Afghanistan,AF,40,484,1,15,3,32,2020-04-10
2,Albania,AL,9,409,1,23,11,165,2020-04-10
3,Algeria,DZ,94,1666,30,235,110,347,2020-04-10
4,American Samoa,AS,0,0,0,0,0,0,2020-04-10
...,...,...,...,...,...,...,...,...,...
241,Wallis and Futuna Islands,WF,0,0,0,0,0,0,2020-04-10
242,Western Sahara,EH,0,4,0,0,0,0,2020-04-10
243,Yemen,YE,0,0,0,0,0,0,2020-04-10
244,Zambia,ZM,0,39,0,1,17,24,2020-04-10


In [8]:
#export to csv

df.to_csv (r'C:\Users\letic\Desktop\Complete_Homework\Project_Two\covid19.csv', index = False, header=True)

## Connect to local database

In [9]:
rds_connection_string = "postgres:postgres@localhost:5432/covid19_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [10]:
engine.table_names()

['covid19', 'covid18']

## Use pandas to load csv converted DataFrame into database

In [12]:
df.to_sql(name='covid20', con=engine, if_exists='append', index=False)